In [1]:
import datetime
import glob
import os
import pandas as pd
import json

In [55]:
def timestamp_of_last_file_in_dir(sfolder, sensor_loc, dir_file_ts):    
    list_of_files = glob.glob(os.path.join(sfolder,'*.dat'))
    latest_file = max(list_of_files, key=os.path.getctime)
    latest_file_ts = datetime.datetime.utcfromtimestamp(os.path.getctime(latest_file))
    dir_file_ts.append({'key_importbackup':sensor_loc,
                        'file_importbackup':latest_file,
                        'ts_importbackup':latest_file_ts}
                      )    
    return dir_file_ts
    
def subfolders(folder):
    subfolders = [f.path for f in os.scandir(folder) if f.is_dir()]
    return subfolders

def timestamp_importbackup(folder):
    dir_file_ts = []
    sensor_locations = subfolders(folder)
    for sensor_location in sensor_locations:
        key_sensor_loc = os.path.basename(sensor_location)
        timestamp_of_last_file_in_dir(sensor_location, key_sensor_loc, dir_file_ts)

        sensor_sublocations = subfolders(sensor_location)
        for sensor_sublocation in sensor_sublocations:
            key_sensor_subloc = '{}_{}'.format(key_sensor_loc, os.path.basename(sensor_sublocation))
            timestamp_of_last_file_in_dir(sensor_sublocation, key_sensor_subloc, dir_file_ts)     

    df_file_ts = pd.DataFrame().from_dict(dir_file_ts)       
    return df_file_ts

def timestamp_status_data(folder_ftpstatus_status, folder_ftpstatus_data):
    
    list_of_status_files = glob.glob(os.path.join(folder_ftpstatus_status,'*STA_*.json'))
    list_of_status_files = [list_of_status_files[0],list_of_status_files[-1]]

    df_status_data_sensor_loc_ts = pd.DataFrame()
    for status_file in list_of_status_files:
        status_loc = os.path.basename(status_file).split('_')

        if len(status_loc) == 4:
            key_status_loc = '{}'.format(status_loc[2])
        elif len(status_loc) == 5:
            key_status_loc = '{}_{}'.format(status_loc[2], status_loc[3])    

        # get creation time status file
        status_file_ts = datetime.datetime.utcfromtimestamp(os.path.getctime(status_file))     

        # get path of status data file
        df_status_status = pd.read_json(status_file, typ='series', orient='values')
        status_file_ts = pd.to_datetime(df_status_status['ts'])
        status_data_file = df_status_status['fn']
        path_status_data_file = os.path.join(folder_ftpstatus_data, 
                                             key_status_loc.replace('_','\\'), 
                                             status_data_file)

        # read status data file
        df_status_data = pd.read_csv(path_status_data_file, skiprows=[0,2,3])    

        sensors = ['ETRO_Status', 'ISA_Status', 'YSI_Status', 'AQD_Status', 'VEC_Status', 'RDI_Status'] 
        s_status_sensors = df_status_data[sensors].isin([-1]).any()
        s_status_sensors.index = [sensor.split('_')[0] for sensor in sensors]    

        df_status_sensors = s_status_sensors.to_frame()
        df_status_sensors.reset_index(inplace=True)
        df_status_sensors.columns = ['data_sensor', 'data_available']
        
        df_status_sensors.loc[:,'file_ftpstatus_data'] = path_status_data_file
        df_status_sensors.loc[:,'file_ftpstatus'] = status_file
        df_status_sensors.loc[:,'key_ftpstatus'] = key_status_loc
        df_status_sensors.loc[:,'ts_ftpstatus'] = status_file_ts    

        df_status_data_sensor_loc_ts = df_status_data_sensor_loc_ts.append(df_status_sensors)     
    
    return df_status_data_sensor_loc_ts  

def error_message(df):
    
    # Check if timedelta is more than 2 hours and data is available
    msg_filter = [all(tup) for tup in zip(df.data_available == True, df.ts_delta > pd.Timedelta(hours=2))]
    df_msg = df

    # add columns for prtg
    df_msg.loc[:, 'channel'] = df_msg['key_ftpstatus']+ ' - ' + df_msg['data_sensor']
    df_msg.loc[:, 'value_error'] = 0
    df_msg.loc[msg_filter, 'value_error'] = 2    
    df_msg.loc[:, 'limitmaxerror'] = 1
    df_msg.loc[:, 'limitmode'] = 1
    df_msg.loc[:, 'limiterrormsg'] = 'Actieve sensor data is succesvol geimporteerd in het DMS'
    df_msg.loc[msg_filter, 'limiterrormsg'] = ['Melding: File {} is op ftp aanwezig maar niet geimporteerd in het DMS'.format(time_of_error) for time_of_error in df.loc[msg_filter, 'file_ftpstatus_data']]

    # filter columns to output
    df_msg_out = df_msg[['channel','value_error','limitmaxerror', 'limitmode','limiterrormsg']]
    df_msg_out.columns = ['channel','value','limitmaxerror', 'limitmode','limiterrormsg']
    
    return df_msg_out

In [56]:
folder_importbackup = r'D:\FEWSProjecten\DMS\ImportBackup\STA'
folder_ftpstatus_status = r'D:\FEWSProjecten\DMS\ftpStatusFiles'
folder_ftpstatus_data = r'D:\FEWSProjecten\DMS\asFromFtpPrimaryroot\STA'
file_monitor_out = r'D:\FEWSProjecten\DMS\monitor_status_import_FEWS.json'

In [75]:
# get dataframe with timestamps status files and status data of sensor-location combinations 
# get dataframe with timestamps of most recent files in FEWS ImportBackup folder
# merge dataframes
# compute timedelta
# create error message
df_ftpstatus_data = timestamp_status_data(folder_ftpstatus_status, folder_ftpstatus_data)
df_importbackup = timestamp_importbackup(folder_importbackup)
df_ftp_backup = df_ftpstatus_data.merge(df_importbackup, how='left',left_on='key_ftpstatus', right_on='key_importbackup')
df_ftp_backup['ts_delta'] = df_ftp_backup['ts_ftpstatus'] - df_ftp_backup['ts_importbackup']
df_msg_out = error_message(df_ftp_backup)

# save to file
error_out = {"prtg":{"result": df_msg_out.to_dict(orient='records')}}
with open(file_monitor_out, 'w') as f:
    print(json.dumps(error_out), file=f)

In [76]:
df_msg_out

,channel,value,limitmaxerror,limitmode,limiterrormsg
0,FL65_A - ETRO,0,1,1,Actieve sensor data is succesvol geimporteerd ...
1,FL65_A - ISA,2,1,1,Melding: File D:\FEWSProjecten\DMS\asFromFtpPr...
2,FL65_A - YSI,0,1,1,Actieve sensor data is succesvol geimporteerd ...
3,FL65_A - AQD,0,1,1,Actieve sensor data is succesvol geimporteerd ...
4,FL65_A - VEC,2,1,1,Melding: File D:\FEWSProjecten\DMS\asFromFtpPr...
5,FL65_A - RDI,0,1,1,Actieve sensor data is succesvol geimporteerd ...
6,FL70 - ETRO,2,1,1,Melding: File D:\FEWSProjecten\DMS\asFromFtpPr...
7,FL70 - ISA,0,1,1,Actieve sensor data is succesvol geimporteerd ...
8,FL70 - YSI,0,1,1,Actieve sensor data is succesvol geimporteerd ...
9,FL70 - AQD,0,1,1,Actieve sensor data is succesvol geimporteerd ...


In [157]:
df_status_sensors.loc[:,'file_ftpstatus'] = status_file
df_status_sensors.loc[:,'key_ftpstatus'] = key_status_loc
df_status_sensors.loc[:,'ts_ftpstatus'] = status_file_ts

In [158]:
df_status_sensors

,data_sensor,data_available,file_ftpstatus,key_ftpstatus,ts_ftpstatus
0,ETRO,True,D:\FEWSProjecten\DMS\ftpStatusFiles\_STA_FL70_...,FL70,2019-05-23 09:56:32.943158
1,ISA,False,D:\FEWSProjecten\DMS\ftpStatusFiles\_STA_FL70_...,FL70,2019-05-23 09:56:32.943158
2,YSI,False,D:\FEWSProjecten\DMS\ftpStatusFiles\_STA_FL70_...,FL70,2019-05-23 09:56:32.943158
3,AQD,False,D:\FEWSProjecten\DMS\ftpStatusFiles\_STA_FL70_...,FL70,2019-05-23 09:56:32.943158
4,VEC,False,D:\FEWSProjecten\DMS\ftpStatusFiles\_STA_FL70_...,FL70,2019-05-23 09:56:32.943158
5,RDI,True,D:\FEWSProjecten\DMS\ftpStatusFiles\_STA_FL70_...,FL70,2019-05-23 09:56:32.943158


In [147]:
df_status_sensors.to_frame().reset_index(inplace=True)


TypeError: to_frame() got an unexpected keyword argument 'inplace'

In [146]:
df_status_sensors

ETRO     True
ISA     False
YSI     False
AQD     False
VEC     False
RDI      True
dtype: bool

In [127]:
df_status_sensors.head()

ETRO     True
ISA     False
YSI     False
AQD     False
VEC     False
dtype: bool

In [126]:
df_status_sensors.to_dict()

{'ETRO': True,
 'ISA': False,
 'YSI': False,
 'AQD': False,
 'VEC': False,
 'RDI': True}

In [104]:
df_status_data[instr_columns].astype(str).isin())

TypeError: only list-like or dict-like objects are allowed to be passed to DataFrame.isin(), you passed a 'str'

In [ ]:
instr_columns

In [70]:

ftp = pd.read_json(os.path.join(path_status_ftp, file), typ='series', orient='values')
m = re.search('(FL.*?)_.*?', ftp['fn'])
pair = {
        'instr':ftp['fn'][0:3],
        'loc':m.group(1),
        'time':pd.to_datetime(ftp['ts'])
        }
ftp_status.append(pair)

,file_ftpstatus,key_ftpstatus,ts_ftpstatus
0,D:\FEWSProjecten\DMS\ftpStatusFiles\_STA_FL65_...,FL65_A,2019-05-23 09:56:32.513159
1,D:\FEWSProjecten\DMS\ftpStatusFiles\_STA_FL65_...,FL65_B,2019-05-23 09:56:32.550159
2,D:\FEWSProjecten\DMS\ftpStatusFiles\_STA_FL65_...,FL65_C,2019-05-23 09:56:32.582159
3,D:\FEWSProjecten\DMS\ftpStatusFiles\_STA_FL65_...,FL65,2019-05-23 09:56:32.626158
4,D:\FEWSProjecten\DMS\ftpStatusFiles\_STA_FL66_...,FL66,2019-05-23 09:56:32.664159
5,D:\FEWSProjecten\DMS\ftpStatusFiles\_STA_FL67_...,FL67,2019-05-23 09:56:32.698159
6,D:\FEWSProjecten\DMS\ftpStatusFiles\_STA_FL68_...,FL68,2019-05-23 09:56:32.736159
7,D:\FEWSProjecten\DMS\ftpStatusFiles\_STA_FL69_...,FL69_A,2019-05-23 09:56:32.777158
8,D:\FEWSProjecten\DMS\ftpStatusFiles\_STA_FL69_...,FL69_B,2019-05-23 09:56:32.815159
9,D:\FEWSProjecten\DMS\ftpStatusFiles\_STA_FL69_...,FL69_C,2019-05-23 09:56:32.858159


In [65]:
status_file

'D:\\FEWSProjecten\\DMS\\ftpStatusFiles\\_STA_FL65_status.json'

datetime.datetime(2019, 5, 23, 9, 56, 32, 626158)

In [ ]:
timestamp_of_file()

In [ ]:
collect_last_file_ts(sensor_location, key_sensor_loc, dir_file_ts)

In [49]:
status_file = list_of_status_files[3]
status_file

'D:\\FEWSProjecten\\DMS\\ftpStatusFiles\\_STA_FL65_status.json'

In [50]:
os.path.basename(os.path.splitext(status_file)[1])

'.json'

In [54]:
status_loc = os.path.basename(status_file).split('_')

In [56]:
len(status_loc)

4

In [53]:
key_status_loc

'FL65_status.json'